In [1]:
!pip install torch torchvision transformers

     |████████████████████████████████| 4.0 MB 8.1 MB/s 
     |████████████████████████████████| 596 kB 81.2 MB/s 
     |████████████████████████████████| 880 kB 69.8 MB/s 
     |████████████████████████████████| 6.6 MB 23.6 MB/s 
     |████████████████████████████████| 77 kB 8.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d6d1b101dd977f0f6aef93f5d4f706cf4c823ae287e5c2ea2a28bda1847002ad
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip3 install emoji==0.6.0

     |████████████████████████████████| 51 kB 4.1 MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49734 sha256=6564c2f1431711fa7ffdcfde6515b77ec8b020a6e6d827e078c48f96b99b828d
  Stored in directory: /root/.cache/pip/wheels/4e/bf/6b/2e22b3708d14bf6384f862db539b044d6931bd6b14ad3c9adc
Successfully built emoji


In [3]:
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, AutoTokenizer
import pandas as pd

class SSTDataset(Dataset):

    def __init__(self, filename, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t')

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        # self.tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base', normalization=True)

        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        #Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'sentence']
        label = self.df.loc[index, 'label']

        #Preprocessing the text to be suitable for BERT
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        tokens = ['[CLS]'] + tokens + ['[SEP]'] #Insering the CLS and SEP token in the beginning and end of the sentence
        if len(tokens) < self.maxlen:
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] #Padding sentences
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] #Prunning the list to be of specified max length

        tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens) #Obtaining the indices of the tokens in the BERT Vocabulary
        tokens_ids_tensor = torch.tensor(tokens_ids) #Converting the list to a pytorch tensor

        #Obtaining the attention mask i.e a tensor containing 1s for no padded tokens and 0s for padded ones
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

In [4]:
from torch.utils.data import DataLoader

#Creating instances of training and development set
#maxlen sets the maximum length a sentence can have
#any sentence longer than this length is truncated to the maxlen size
train_set = SSTDataset(filename = 'train.tsv', maxlen = 500)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
dev_set = SSTDataset(filename = 'dev.tsv', maxlen = 500)

#Creating intsances of training and development dataloaders
train_loader = DataLoader(train_set, batch_size = 8, num_workers = 2)
dev_loader = DataLoader(dev_set, batch_size = 8, num_workers = 2)

print("Done preprocessing training and development data.")

Done preprocessing training and development data.


In [6]:
import torch
import torch.nn as nn
from transformers import BertModel, AutoModel

class SentimentClassifier(nn.Module):

    def __init__(self):
        super(SentimentClassifier, self).__init__()
        #Instantiating BERT model object 
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        # self.bert_layer = AutoModel.from_pretrained("vinai/bertweet-base")

        
        #Classification layer
        #input dimension is 768 because [CLS] embedding has a dimension of 768
        #output dimension is 1 because we're working with a binary classification problem
        self.cls_layer = nn.Linear(768, 1)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        outputs = self.bert_layer(seq, attention_mask = attn_masks)
        cont_reps = outputs.last_hidden_state

        #Obtaining the representation of [CLS] head (the first token)
        cls_rep = cont_reps[:, 0]

        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(cls_rep)

        return logits

In [7]:
gpu = 0 #gpu ID

print("Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...")
net = SentimentClassifier()
net.cuda(gpu) #Enable gpu support for the model
print("Done creating the sentiment classifier.")

Creating the sentiment classifier, initialised with pretrained BERT-BASE parameters...


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Done creating the sentiment classifier.


In [8]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [9]:
import time

def train(net, criterion, opti, train_loader, dev_loader, max_eps, gpu):

    best_acc = 0
    st = time.time()
    for ep in range(max_eps):
        
        net.train()
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            #Clear gradients
            opti.zero_grad()  
            #Converting these to cuda tensors
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)

            #Obtaining the logits from the model
            logits = net(seq, attn_masks)

            #Computing loss
            loss = criterion(logits.squeeze(-1), labels.float())

            #Backpropagating the gradients
            loss.backward()

            #Optimization step
            opti.step()
              
            if it % 100 == 0:
                
                acc = get_accuracy_from_logits(logits, labels)
                print("Iteration {} of epoch {} complete. Loss: {}; Accuracy: {}; Time taken (s): {}".format(it, ep, loss.item(), acc, (time.time()-st)))
                st = time.time()

        
        dev_acc, dev_loss = evaluate(net, criterion, dev_loader, gpu)
        print("Epoch {} complete! Development Accuracy: {}; Development Loss: {}".format(ep, dev_acc, dev_loss))
        if dev_acc > best_acc:
            print("Best development accuracy improved from {} to {}, saving model...".format(best_acc, dev_acc))
            best_acc = dev_acc
            torch.save(net.state_dict(), 'sstcls_{}.dat'.format(ep))

In [10]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

def evaluate(net, criterion, dataloader, gpu):
    net.eval()

    mean_acc, mean_loss = 0, 0
    count = 0

    with torch.no_grad():
        for seq, attn_masks, labels in dataloader:
            seq, attn_masks, labels = seq.cuda(gpu), attn_masks.cuda(gpu), labels.cuda(gpu)
            logits = net(seq, attn_masks)
            mean_loss += criterion(logits.squeeze(-1), labels.float()).item()
            mean_acc += get_accuracy_from_logits(logits, labels)
            count += 1

    return mean_acc / count, mean_loss / count

In [ ]:
num_epoch = 5
#fine-tune the model
train(net, criterion, opti, train_loader, dev_loader, num_epoch, gpu)

In [12]:
c = 0
for it, (seq, attn_masks, labels) in enumerate(train_loader):
  c=c+1
print(c)


226


In [13]:
class TestDataset(Dataset):

    def __init__(self, filename, maxlen):

        #Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter = '\t')

        #Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        # self.tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base', normalization=True)
        
        # the input length for BERT model. Max length is 512
        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):

        #Selecting the content and label at the specified index in the data frame
        sentence = self.df.loc[index, 'sentence']
        
        # Tokenize the tweet and insering the CLS and SEP
        tokens = self.tokenizer.tokenize(sentence) #Tokenize the sentence
        
        if len(tokens) < self.maxlen:
            #Padding token
            tokens = tokens + ['[PAD]' for _ in range(self.maxlen - len(tokens))] 
        else:
            tokens = tokens[:self.maxlen-1] + ['[SEP]'] 
        
        #Converting the token to a pytorch ID tensor
        tokens_ids_tensor = torch.tensor(self.tokenizer.convert_tokens_to_ids(tokens)) 

        #Obtaining the attention mask
        attn_mask = (tokens_ids_tensor != 0).long()
        
        return tokens_ids_tensor, attn_mask

In [15]:
dev_set_2 = TestDataset(filename = 'dev.tsv', maxlen = 300)
dev_loader_pre = DataLoader(dev_set_2, batch_size = 1, num_workers = 0)
print("Done preprocessing test data.")

Done preprocessing test data.


In [ ]:
test_set = TestDataset(filename = 'test.tsv', maxlen = 300)
test_loader = DataLoader(test_set, batch_size = 1, num_workers = 0)
print("Done preprocessing test data.")

Done preprocessing test data.


In [16]:
def predict(net, test_loader, weight_file):
    # load weight
    net.load_state_dict(torch.load(weight_file))
    
    predictions = []
    classes = ["0", "1"]
    
    # Predict process
    with torch.no_grad():
        for seq, attn_masks in test_loader:
            seq, attn_masks = seq.cuda(gpu), attn_masks.cuda(gpu)
            logits = net(seq, attn_masks)
            probs = torch.sigmoid(logits.unsqueeze(-1))
            soft_probs = (probs > 0.5).long()
            predictions.append(classes[soft_probs.cpu().numpy().squeeze()])
    
    return predictions

In [18]:
weight_file = r"sstcls_0.dat"
predictions = predict(net, dev_loader_pre, weight_file)
#predictions = predict(net, test_loader, weight_file)

In [30]:
filename = 'dev.tsv'

dev_label_df = pd.read_csv(filename, delimiter = '\t')
dev_label = dev_label_df.label  #.astype("category")

In [34]:
pre_labels = []
for i in predictions:
  pre_labels.append(int(i))

In [19]:
import sklearn.metrics as metrics

In [38]:
print(metrics.accuracy_score(dev_label, pre_labels))
print(metrics.precision_score(dev_label, pre_labels))
print(metrics.recall_score(dev_label, pre_labels))
print(metrics.f1_score(dev_label, pre_labels, average='macro'))
print(metrics.roc_auc_score(dev_label, pre_labels))

0.8487394957983193
0.8051948051948052
0.45255474452554745
0.7436130687911751
0.7099018264112454


In [ ]:
with open('test.predictions.txt', 'w') as output:
  output.write('Id,Predicted\n')
  counter = 0
  for elem in predictions:
    output.write(str(counter) + ',' + elem+'\n')
    counter+=1

In [ ]:
c = 0
for seq, attn_masks in test_loader:
  c +=1
print(c)

558
